#  Combining Garmin and Clue data.
This script uses Garmin and Clue data to visualise running performance at the same time as menstrual data. Although Garmin has it's own menstrual calendar it is not possible to download this data or to see this data alongside performance information. If you want to track your own menstrual data rather than using an app create a csv file with a column of the dates you've had your period instead.

You need three files for this script to run.
* `Activities.csv` The CSV downloaded from Garmin connect when all activities are selected.
* `Activities_running_only.csv` The CSV downloaded from Garmin connect when only running is slected. This will export more of the running statistics.
* `clue_measurements.json` This is the `measurements.json` file created when you download data from the menstrual tracking app Clue. If you aren't using clue then you will need `period_dates.csv`, enter the dates in the format `YYYY-MM-DD`.

To download from Garmin:
1. Login to Garmin Connect.
2. Sidebar select `Activities>All activities`.  
3. Select either running or all activites. You will need both.
4. Scroll down as far as the dates you want to include.
5. Click `Export csv`.
6. Move your csv file to the same place you've chosen your data (probably a directory where this script is saved.)

To download from Clue:
1. Open the Clue app
2. Go to the More Menu (the = in the top-right corner of your Cycle View)
3. Tap Settings
4. Tap Download my data
5. Tap Request data
6. A screen will appear with a unique password to download the data file - copy this. You will probably want to send this to yourself as it's likely you'll run this script on a computer rather than phone. So paste->send->copy again.
7. Open the email from Clue that was sent to your Clue email address
8. The email will include a link to download the data file, which expires after 72 hours
9. Tap Download data
10. Extract the zip to the desired location.
11. When you open the file enter the password. Save the `measurements` file as `clue_measurements`. The others can now be deleted.

In [2]:
import pandas as pd
import altair as alt
import numpy as np
import datetime as dt
from IPython.display import display, Markdown
from scipy import stats
from datetime import timedelta
import json
import os

# Your inputs
Input your HR zones and the name of the directory containing your data.

## Heart rate zones
To access your heart rate (HR) zones in the Garmin Connect app, you can do the following:
1. Open the app
2. Select More in the bottom right corner
3. Select Garmin Devices
4. Select your device
5. Select User Settings, User Profile, or My Stats
6. Select Heart Rate Zones or Heart Rate
7. Customize your HR zones
8. Select Done

## Data directory
* If your data directory is saved in the same place as this script then you can enter the name inside \" \". Include "/" at the end.
* Prefix the directory with "../" if it's in the directory above. Use this as many times as you need to go up. 
* If your data isn't in a separate folder and is in the same place as the script then set to \"\".
* If your data is in a completely different part of your system you can use an absolute path for example \"C:/User/name/data/", this is NOT advised if you are going to be sharing this script. Include "/" at the end.

In [3]:
hr_zones = [120,148, 165, 176, 185, 213]
dir_name = "my_data/" # The name and path of the directory you have the data saved in.

# Setup the Functions
## Plotting functions

In [4]:
display(Markdown("Function - scatter plot without period data but with upper and lower bounds."))
def scatter_plot_no_p(data_frame, col1, col2, low = 0, high = 1.0):
    # Plot the points labelled Success first then others
    data_crop = data_frame[[col1, col2]].dropna(how = "any")
    data_crop["Band"] = None
    data_crop.loc[data_crop[col2]< low, "Band"] = "low"
    data_crop.loc[data_crop[col2]> high, "Band"] = "high"
    data_crop.loc[(data_crop[col2]>=low) & (data_crop[col2]<= high), "Band"] = "Mid"
    if col1 == "Date": xvar = col1+":T"
    else: xvar = col1+":Q"
    points = alt.Chart(data_crop).mark_square(size=20).encode(
            x=xvar,
            y=alt.Y(col2+":Q").scale(zero=False),
            color = 'Band'+":N",
            tooltip = [col1, col2]
        ).properties(
    width=800,
    height=300
).interactive()
    line = alt.Chart(data_crop).mark_line(
            color='blue',
            size=1
        ).transform_window(
            rolling_mean='mean('+col2+')',
            frame=[-5, 5]
        ).encode(
        x = xvar,
        y = alt.Y('rolling_mean:Q').scale(zero=False)
    ).properties(
    width=800,
    height=300
)
    return(points+line)


Function - scatter plot without period data but with upper and lower bounds.

In [5]:
def scatter_plot(data_frame, col1, col2):
    # Plot the points labelled Success first then others
    data_crop = data_frame[[col1, col2, "Period"]].dropna(how = "any")
    if col1 == "Date": xvar = col1+":T"
    else: xvar = col1+":Q"
    points = alt.Chart(data_crop).mark_circle(size=20).encode(
            x=xvar,
            y=alt.Y(col2+":Q").scale(zero=False),
            color='Period'+":N",
            tooltip = [col1, col2, "Period"]
        ).properties(
    width=800,
    height=300
).interactive()
    line = alt.Chart(data_crop).mark_line(
            color='blue',
            size=1
        ).transform_window(
            rolling_mean='mean('+col2+')',
            frame=[-5, 5]
        ).encode(
        x = xvar,
        y = alt.Y('rolling_mean:Q').scale(zero=False)
    ).properties(
    width=800,
    height=300
)
    return(points+line)

## Function for numbering Heartrate zones

In [6]:
def set_hr_zone(row, hr_zones):
    hr = row["Avg HR"]
    if hr <hr_zones[0]: return 1
    elif hr_zones[0] <= hr < hr_zones[1]: return 2
    elif hr_zones[1] <= hr < hr_zones[2]: return 3
    elif hr_zones[2]<= hr < hr_zones[3]: return 4
    elif hr_zones[3] <= hr < hr_zones[4]: return 5
    elif hr_zones[4]<= hr < hr_zones[5]: return 6
    elif hr_zones[5]<= hr : return 6
    else: return 1

## Function to convert `Elapsed Time` and `Duration` to integer number of seconds.

In [7]:
def make_del(row):
    if "Elapsed Time" in row.index:
        entry = row["Elapsed Time"]
    elif "Duration" in row.index:
        entry = row["Duration"]
    else: raise("Duration column is missing, check csv files.")
    splits = entry.split(':')
    if len(splits)>2:
        h, m, s = splits
        if "." in s:
          s, _ = s.split('.')
    elif len(splits)==2:
        h = 0
        m = splits[0]
        if "." in s:
          s, _ = s.split('.')
    else: return dt.timedelta(hours=int(0), minutes=int(0), seconds=int(0)).total_seconds()
    return dt.timedelta(hours=int(h), minutes=int(m), seconds=int(s)).total_seconds()

# Load Data
## Load the activities data

In [8]:
try: os.path.isfile(dir_name+ "Activities.csv")
except: print("Activities.csv is not in the data directory. Check location and filename")
activities_df = pd.read_csv(dir_name+"Activities.csv", header = 0, parse_dates=["Date"])
activities_df.head(-5)

,Activity Type,Date,Favorite,Title,Distance,Calories,Time,Avg HR,Max HR,Aerobic TE,...,Max Resp,Stress Change,Stress Start,Stress End,Avg Stress,Max Stress,Moving Time,Elapsed Time,Min Elevation,Max Elevation
0,Running,2024-07-21 08:00:25,False,Bath and North East Somerset Running,15.83,"1,030",01:42:18,176,188,4.4,...,41,--,--,--,--,--,01:40:13,02:02:12,15,186
1,Indoor Cycling,2024-07-17 20:59:15,False,Indoor Cycling,0.00,58,00:20:06,88,124,0.2,...,17,--,--,--,--,--,00:00:00,00:20:06,--,--
2,Indoor Cycling,2024-07-17 17:59:00,False,Indoor Cycling,0.00,427,00:45:58,145,182,3.3,...,43,--,--,--,--,--,00:00:00,00:45:58,--,--
3,Cycling,2024-07-17 17:32:09,False,Bath and North East Somerset Cycling,4.75,131,00:20:13,117,145,1.1,...,--,--,--,--,--,--,00:18:22,00:20:13,25,51
4,Cycling,2024-07-17 11:34:19,False,Bath and North East Somerset Cycling,4.12,103,00:18:21,108,141,0.5,...,--,--,--,--,--,--,00:17:31,03:00:05,26,53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590,Walking,2023-08-22 17:03:03,False,Maldon Walking,2.61,155,00:40:42,81,108,0.4,...,28,--,--,--,--,--,00:37:58,00:40:42,22,35
591,Walking,2023-08-20 19:16:42,False,Maldon Walking,1.01,72,00:17:25,100,131,0.3,...,29,--,--,--,--,--,00:15:02,00:18:34,18,36
592,Running,2023-08-20 17:41:19,False,Maldon Running,9.52,662,01:27:29,141,166,3.0,...,37,--,--,--,--,--,01:19:33,01:27:29,15,63
593,Stand Up Paddleboarding,2023-08-19 13:56:30,False,Maldon Stand Up Paddleboarding,2.67,279,00:58:16,111,135,1.0,...,--,--,--,--,--,--,00:42:36,00:58:16,--,--


## Load the running only activities data

In [9]:
try: os.path.isfile(dir_name+ "Activities_running_only.csv")
except: raise("Activities_running_only.csv is not in the data directory. Check location and filename")
running_df = pd.read_csv(dir_name+"Activities_running_only.csv", header = 0, parse_dates=["Date"])
running_df.head(-5)

,Activity Type,Date,Favorite,Title,Distance,Calories,Time,Avg HR,Max HR,Aerobic TE,...,Best Lap Time,Number of Laps,Max Temp,Avg Resp,Min Resp,Max Resp,Moving Time,Elapsed Time,Min Elevation,Max Elevation
0,Running,2024-07-21 08:00:25,False,Bath and North East Somerset Running,15.83,"1,030",01:42:18,176,188,4.4,...,00:04:53.2,16,29.0,34,17,41,01:40:13,02:02:12,15,186
1,Running,2024-07-16 18:37:07,False,Bath and North East Somerset Running,4.75,287,00:37:30,138,169,2.3,...,00:05:50.6,5,31.0,26,16,34,00:30:21,00:39:24,23,50
2,Running,2024-07-14 08:14:53,False,Bath and North East Somerset Running,8.79,527,00:54:05,155,170,3.3,...,00:04:31.6,9,28.0,26,16,35,00:52:48,01:15:26,12,37
3,Running,2024-07-12 19:16:48,False,Wiltshire Running,9.66,620,00:57:05,169,188,3.6,...,00:03:54.3,10,23.0,32,21,41,00:56:59,00:57:05,52,158
4,Running,2024-07-11 19:30:12,False,Bath and North East Somerset Running,4.38,297,00:34:38,148,171,2.3,...,00:02:05.8,5,28.0,26,11,35,00:34:22,00:35:27,34,189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,Running,2023-08-18 09:03:35,False,Maldon Running,1.48,103,00:11:27,147,163,2.0,...,00:03:39.9,2,30.0,32,14,37,00:11:23,00:11:27,27,37
211,Running,2023-08-15 18:49:21,False,Maldon - HR efforts,5.87,395,00:43:24,155,179,3.1,...,00:00:51.0,12,29.0,31,21,38,00:43:22,00:43:24,15,27
212,Running,2023-08-13 11:51:14,False,Bath and North East Somerset - 12 miles + WU +...,23.71,"1,607",03:00:43,164,183,4.6,...,00:00:15.1,37,31.0,33,21,43,03:00:16,03:00:51,19,44
213,Running,2023-08-11 09:01:31,False,Bath and North East Somerset - Base,5.02,342,00:38:49,147,165,2.6,...,00:01:49.4,6,30.0,29,20,37,00:38:18,00:39:01,30,55


## Load period data from Clue - JSON

In [10]:
try: period_df = pd.read_json(dir_name + "clue_measurements.json")
except: 
    try: period_df = pd.read_csv(dir_name + "period_dates.csv")
    except: print("No period data found. is not in the data directory. Check location and filename")
period_df = period_df[period_df.type=='period'].sort_values(by="date")
period_dates=period_df.date

## Set the data-type for the Date columns

In [11]:
activities_df["Date"] = pd.to_datetime(activities_df["Date"], dayfirst= True)
running_df["Date"] = pd.to_datetime(running_df["Date"], dayfirst= True)
period_dates = pd.to_datetime(period_dates, dayfirst= True)

# Combine and restucture the data
## Add period data to activities and running dataframes.

In [12]:
running_df['Period'] = 0
activities_df['Period'] = 0
running_df.loc[running_df.Date.dt.date.isin(period_dates.dt.date),'Period'] = 1
activities_df.loc[activities_df.Date.dt.date.isin(period_dates.dt.date),'Period'] = 1


## Get the running data
### Extract data from the strings.

In [13]:
running_df.rename(columns = {"Time": "Duration"}, inplace= True)
running_df["Distance"] = pd.to_numeric(running_df["Distance"])
running_df[['L_GCT Balance', 'R_GCT Balance']] = running_df["Avg GCT Balance"].str.split(" / ", expand= True)
running_df.loc[:,"L_GCT Balance"] = running_df.loc[:,"L_GCT Balance"].str.replace(r'\D', '', regex=True)
running_df.loc[:, "R_GCT Balance"] = running_df.loc[:,"L_GCT Balance"].str.replace(r'\D', '', regex=True)
col_list = ['Avg Stride Length', 'Avg Vertical Ratio', 'Avg Vertical Oscillation', 'Avg Run Cadence', 
            'Avg Ground Contact Time', "L_GCT Balance", "R_GCT Balance"]
running_df[col_list] = running_df[col_list].apply(pd.to_numeric, errors='coerce')
running_df["L_GCT Balance"] = running_df["L_GCT Balance"] / 1000
running_df["R_GCT Balance"] = running_df["R_GCT Balance"] / 1000
running_df["Date"] = pd.to_datetime(running_df["Date"], dayfirst=True, errors = "coerce")
running_df.head()

,Activity Type,Date,Favorite,Title,Distance,Calories,Duration,Avg HR,Max HR,Aerobic TE,...,Avg Resp,Min Resp,Max Resp,Moving Time,Elapsed Time,Min Elevation,Max Elevation,Period,L_GCT Balance,R_GCT Balance
0,Running,2024-07-21 08:00:25,False,Bath and North East Somerset Running,15.83,"1,030",01:42:18,176,188,4.4,...,34,17,41,01:40:13,02:02:12,15,186,0,0.507,0.507
1,Running,2024-07-16 18:37:07,False,Bath and North East Somerset Running,4.75,287,00:37:30,138,169,2.3,...,26,16,34,00:30:21,00:39:24,23,50,0,0.500,0.500
2,Running,2024-07-14 08:14:53,False,Bath and North East Somerset Running,8.79,527,00:54:05,155,170,3.3,...,26,16,35,00:52:48,01:15:26,12,37,0,0.506,0.506
3,Running,2024-07-12 19:16:48,False,Wiltshire Running,9.66,620,00:57:05,169,188,3.6,...,32,21,41,00:56:59,00:57:05,52,158,0,0.519,0.519
4,Running,2024-07-11 19:30:12,False,Bath and North East Somerset Running,4.38,297,00:34:38,148,171,2.3,...,26,11,35,00:34:22,00:35:27,34,189,1,0.498,0.498


## Collect activities by date
### Edit to match your own heart rate zones

In [14]:
start_date = min(running_df.Date.min(),activities_df.Date.min())

### Calculate the total duration in seconds for further calculations.

In [15]:
running_df["Duration_seconds"]=running_df.apply(make_del, axis=1)
activities_df["Duration_seconds"]=activities_df.apply(make_del, axis=1)

### Set the heartrate zones

In [16]:
activities_df["Avg HR"] = pd.to_numeric(activities_df["Avg HR"], errors='coerce')
activities_df["hr_zone"] = activities_df.apply(set_hr_zone, hr_zones = hr_zones, axis = 1)

### Day totals
Calculate the total activity per day and the training load corresponding.
Trianing load calculated using: $$\text{load} = \text{minutes of activity} * \text{average heart rate}.$$

In [98]:

#running_df.Date.dt.normalize(),start_date.date()
date_format = "%Y/%m/%d"
ndays = (dt.datetime.today()-start_date).days
by_date = {}
for i in range(ndays):
    new_date = (start_date+timedelta(days=i))
    next_date = (start_date+timedelta(days=i+7))
    runs = running_df.loc[running_df.Date.dt.date==pd.Timestamp(new_date.date()).date()]
    activities = activities_df[activities_df.Date.dt.date==pd.Timestamp(new_date.date()).date()]
    next_week = pd.date_range(new_date, next_date, periods = 7)
    if sum(period_dates==pd.Timestamp(new_date.date())):
        period = 1
    elif period_dates.isin(next_week.date).sum()>0:
        period = 2
    else:
        period = 0
    if len(activities)==0:
        tot_dist=0
        duration=0
        load=0
    else:
        tot_dist = runs["Distance"].sum()
        duration = dt.timedelta(seconds=activities["Duration_seconds"].sum())
        load = (activities["Duration_seconds"]%60 * activities.hr_zone).sum()
    running_df.loc[running_df.Date.dt.date==pd.Timestamp(new_date.date()).date(),"Period"] = period
    activities_df.loc[activities_df.Date.dt.date==pd.Timestamp(new_date.date()).date(),"Period"] = period
    by_date[i] = {
        'Date': new_date,
        'run_dist': tot_dist,
        'duration': duration,
        'duration_seconds': activities["Duration_seconds"].sum(),
        'load': load,
        'Period': period}
overall_by_date_df = pd.DataFrame(by_date).T
overall_by_date_df.Date = pd.to_datetime(overall_by_date_df.Date)
overall_by_date_df.run_dist = pd.to_numeric(overall_by_date_df.run_dist)
overall_by_date_df.duration_seconds = pd.to_numeric(overall_by_date_df.duration_seconds)
overall_by_date_df.load = pd.to_numeric(overall_by_date_df.load)
overall_by_date_df.head()

C:\Users\es21851\AppData\Local\Temp\ipykernel_26728\1883445181.py:13: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  elif period_dates.isin(next_week.date).sum()>0:
C:\Users\es21851\AppData\Local\Temp\ipykernel_26728\1883445181.py:13: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  elif period_dates.isin(next_week.date).sum()>0:
C:\Users\es21851\AppData\Local\Temp\ipykernel_26728\1883445181.py:13: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Expl

,Date,run_dist,duration,duration_seconds,load,Period
0,2023-07-22 09:01:52,0.0,0,0.0,0,1
1,2023-07-23 09:01:52,0.0,0,0.0,0,1
2,2023-07-24 09:01:52,0.0,0,0.0,0,0
3,2023-07-25 09:01:52,0.0,0,0.0,0,0
4,2023-07-26 09:01:52,0.0,0,0.0,0,0


## Find the relationship between training load and run distance
By finding the most common relationship between load and distance we can then create a proxy distance for none-running activities. We can then use this to create `proxy-distance` to aid coming back from injury. This also prevents over-training from additional activities.

If there are more than 2 weeks with zero running then mileage will be adjusted based on previous running mileage. The `proxy-distance` will then be used to generate an alternative activity that will slowly reduce and readjust to running again. 

In [99]:
overall_by_date_df["coef"]=overall_by_date_df["run_dist"]/overall_by_date_df["load"]
predict_coef = overall_by_date_df["coef"].median()
overall_by_date_df["proxy_kms"] = overall_by_date_df["load"]*predict_coef

## Plot the training load


In [100]:
training_load = scatter_plot(overall_by_date_df, "Date", "load")
(training_load)

alt.LayerChart(...)

In [101]:
display(Markdown("## Calculate the rolling totals and percentages."))
overall_by_date_df["Proxy KMs - Rolling weekly AVG"] = overall_by_date_df.proxy_kms.rolling(window = 7, axis=0).mean()
overall_by_date_df["Run KMs - Rolling weekly AVG"] = overall_by_date_df.run_dist.rolling(window = 7, axis=0).mean()
overall_by_date_df["Time - Rolling weekly AVG"] = overall_by_date_df.duration_seconds.rolling(window = 7, axis=0).mean()
overall_by_date_df["Load - Rolling weekly AVG"] = overall_by_date_df.load.rolling(window = 7, axis=0).mean()
overall_by_date_df.head()

    

## Calculate the rolling totals and percentages.

C:\Users\es21851\AppData\Local\Temp\ipykernel_26728\3106965676.py:2: FutureWarning: The 'axis' keyword in Series.rolling is deprecated and will be removed in a future version. Call the method without the axis keyword instead.
  overall_by_date_df["Proxy KMs - Rolling weekly AVG"] = overall_by_date_df.proxy_kms.rolling(window = 7, axis=0).mean()
C:\Users\es21851\AppData\Local\Temp\ipykernel_26728\3106965676.py:3: FutureWarning: The 'axis' keyword in Series.rolling is deprecated and will be removed in a future version. Call the method without the axis keyword instead.
  overall_by_date_df["Run KMs - Rolling weekly AVG"] = overall_by_date_df.run_dist.rolling(window = 7, axis=0).mean()
C:\Users\es21851\AppData\Local\Temp\ipykernel_26728\3106965676.py:4: FutureWarning: The 'axis' keyword in Series.rolling is deprecated and will be removed in a future version. Call the method without the axis keyword instead.
  overall_by_date_df["Time - Rolling weekly AVG"] = overall_by_date_df.duration_sec

,Date,run_dist,duration,duration_seconds,load,Period,coef,proxy_kms,Proxy KMs - Rolling weekly AVG,Run KMs - Rolling weekly AVG,Time - Rolling weekly AVG,Load - Rolling weekly AVG
0,2023-07-22 09:01:52,0.0,0,0.0,0,1,NaN,0.0,NaN,NaN,NaN,NaN
1,2023-07-23 09:01:52,0.0,0,0.0,0,1,NaN,0.0,NaN,NaN,NaN,NaN
2,2023-07-24 09:01:52,0.0,0,0.0,0,0,NaN,0.0,NaN,NaN,NaN,NaN
3,2023-07-25 09:01:52,0.0,0,0.0,0,0,NaN,0.0,NaN,NaN,NaN,NaN
4,2023-07-26 09:01:52,0.0,0,0.0,0,0,NaN,0.0,NaN,NaN,NaN,NaN


In [102]:
training_load_rolling = scatter_plot(overall_by_date_df, "Date", "Load - Rolling weekly AVG")
time_rolling = scatter_plot(overall_by_date_df, "Date", "Time - Rolling weekly AVG")
run_kms_rolling = scatter_plot(overall_by_date_df, "Date", "Run KMs - Rolling weekly AVG")
proxy_kms_rolling = scatter_plot(overall_by_date_df, "Date", "Proxy KMs - Rolling weekly AVG")
(training_load_rolling & time_rolling) | (run_kms_rolling & proxy_kms_rolling )

alt.HConcatChart(...)

## Calculate the percentage of the weekly distance and the increase from the previous week.

In [103]:
overall_by_date_df["Run KMs - Rolling total"] = 7*overall_by_date_df["Run KMs - Rolling weekly AVG"]
overall_by_date_df["Percent_distance_tot"] = overall_by_date_df.run_dist/ (7*overall_by_date_df["Run KMs - Rolling weekly AVG"])
overall_by_date_df["Distance_percent_increase"] = overall_by_date_df["Run KMs - Rolling total"].diff(periods=7)

In [104]:
display(Markdown("## Plot the distances and percentages"))
upper_bound = 0.33
lower_bound = 0.1
date_percent_dist = scatter_plot_no_p(overall_by_date_df, "Date", "Percent_distance_tot", lower_bound, upper_bound)
date_rolling_dist = scatter_plot_no_p(overall_by_date_df, "Date", "Run KMs - Rolling weekly AVG", 30, 80)
date_dist_incre = scatter_plot_no_p(overall_by_date_df, "Date", "Distance_percent_increase", 0, 0.1)

## Plot the distances and percentages

In [105]:
(date_percent_dist) & (date_rolling_dist) & (date_dist_incre) 

alt.VConcatChart(...)

## Visualise running performance stats with period markup

In [106]:
date_stride_leng = scatter_plot(running_df, "Date", 'Avg Stride Length')
date_vert_rat = scatter_plot(running_df, "Date", 'Avg Vertical Ratio')
date_cadence = scatter_plot(running_df, "Date", 'Avg Run Cadence')
date_vert_osc = scatter_plot(running_df, "Date", 'Avg Vertical Oscillation')
date_grd_cont = scatter_plot(running_df, "Date", 'Avg Ground Contact Time')
date_l_balance = scatter_plot(running_df, "Date","L_GCT Balance")
date_r_balance = scatter_plot(running_df, "Date","R_GCT Balance")

(date_stride_leng | date_vert_rat) & (date_cadence | date_vert_osc) & (date_grd_cont ) & (date_l_balance | date_r_balance)

alt.VConcatChart(...)